Process the data and load them into a database, so we can query them later easily. 

In [130]:
import pandas as pd
import pickle
import numpy as np
import sqlalchemy as sql
import csv

In [120]:
id_to_title = {}
with open('data/labels_apr.csv', 'r') as csvfile:
    next(csvfile) # skip header
    csv_reader = csv.reader(csvfile, delimiter=',', )
    for row in csv_reader:
        id_to_title[row[0]] = row[1]

In [122]:
data = []
with open('data/counts_apr.pkl', 'rb') as f:
    data = pickle.load(f)    

In [178]:
engine = sql.create_engine('sqlite:///database.db')
metadata = sql.MetaData()

articles = sql.Table('articles', metadata,
    sql.Column('id', sql.String),
    sql.Column('title', sql.String),
    sql.Column('year', sql.Integer),
    sql.Column('month', sql.Integer),
    sql.Column('day', sql.Integer),
    sql.Column('view_count', sql.Integer),
    sql.Column('main_category', sql.String)
)
metadata.create_all(engine)

In [180]:
def print_rows(limit=10):
    i = 0
    with engine.connect() as con:
        rows = con.execute(articles.select())
        for r in rows:
            print(int(r.view_count))
            i += 1
            if ( i == limit): return

In [181]:
print_rows(10)

0
0
0
0
0
0
0
0
0
0


In [182]:
processed_data = []
month_to_index = [
  dict(month='10', year='2014', start=193, end=936),
  dict(month='11', year='2014', start=937, end=1656),
  dict(month='12', year='2014', start=1657, end=2400),
  dict(month='01', year='2015', start=2401, end=3144),
  dict(month='02', year='2015', start=3145, end=3816),
  dict(month='03', year='2015', start=3817, end=4560),
  #dict(month='04', year='2015', start=4561, end=5279)
]

with engine.connect() as con:
    for wiki_id in list(data)[:1000|:
        hours = data[wiki_id]
        if str(wiki_id) in id_to_title:
            for each in month_to_index:
                days = int((each['end'] - each['start'] + 1) / 24)
                array = np.array(hours[each['start'] - 1 : each['end']])
                day_total = np.sum(array.reshape((days, 24)), axis=1)
                for each_day in range(days):
                    title = id_to_title[str(wiki_id)]
                    con.execute(articles.insert().values(
                        id = wiki_id, 
                        title = title, 
                        year = each['year'], 
                        month = each['month'], 
                        day = each_day, 
                        view_count = int(day_total[each_day])
                    ))
                    

## Explore Wikipedia data

In [185]:
import wikipedia

In [189]:
y = wikipedia.page('Rose_McIver')

In [192]:
y.categories

['1988 births',
 '20th-century New Zealand actresses',
 '21st-century New Zealand actresses',
 'All Wikipedia articles written in New Zealand English',
 'Articles with hCards',
 'CS1 errors: dates',
 'Living people',
 'New Zealand child actresses',
 'New Zealand expatriates in the United States',
 'New Zealand film actresses',
 'New Zealand people of Scottish descent',
 'New Zealand television actresses',
 'Pages with citations having bare URLs',
 'Pages with citations lacking titles',
 'People educated at Avondale College',
 'People from Auckland',
 'University of Auckland alumni',
 'Use New Zealand English from August 2012',
 'Use dmy dates from February 2018',
 'Webarchive template wayback links',
 'Wikipedia articles with BNF identifiers',
 'Wikipedia articles with GND identifiers',
 'Wikipedia articles with ISNI identifiers',
 'Wikipedia articles with LCCN identifiers',
 'Wikipedia articles with VIAF identifiers']